In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import mean_squared_error

from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from subprocess import check_output

Using TensorFlow backend.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
data = {
    'tra': pd.read_csv('data/air_visit_data.csv'),
    'as': pd.read_csv('data/air_store_info.csv'),
    'hs': pd.read_csv('data/hpg_store_info.csv'),
    'ar': pd.read_csv('data/air_reserve.csv'),
    'hr': pd.read_csv('data/hpg_reserve.csv'),
    'id': pd.read_csv('data/store_id_relation.csv'),
    'tes': pd.read_csv('data/sample_submission.csv'),
    'hol': pd.read_csv('data/date_info.csv').rename(columns={'calendar_date':'visit_date'})
    }
# only work with air id and all restaurants available on both systems
data['hr'] = pd.merge(data['hr'], data['id'], how='inner', on=['hpg_store_id'])
data['hr'] = data['hr'].drop('hpg_store_id', axis=1)
data['ar'] = data['ar'].append(data['hr'])

In [47]:
# Create single data set with all relevant base data:
data['tra']['visit_datetime'] = pd.to_datetime(data['tra']['visit_date'])
data['tra']['visit_date']     = data['tra']['visit_datetime'].dt.date
data['ar']['res_visit_datetime'] = pd.to_datetime(data['ar']['visit_datetime'])
data['ar']['reserve_datetime']   = pd.to_datetime(data['ar']['reserve_datetime'])
data['ar']['visit_date']         = data['ar']['res_visit_datetime'].dt.date
data['ar']['reserve_diff'] = data['ar'].apply(lambda r: (r['res_visit_datetime']
                                                         - r['reserve_datetime']).days, 
                                        axis=1)
data['ar'].drop('visit_datetime',  axis=1, inplace=True)
data['ar'].drop('reserve_datetime',  axis=1, inplace=True)
data['ar'].drop('res_visit_datetime',  axis=1, inplace=True)
avg_reserv = data['ar'].groupby(['air_store_id','visit_date'], 
        as_index=False).mean().reset_index().drop('index', axis=1)
data['ar'] = data['ar'].groupby(['air_store_id','visit_date'], 
                                as_index=False).sum().reset_index()
data['ar'] = data['ar'].drop(['reserve_diff'],axis=1)
data['ar'] = data['ar'].drop(['index'],axis=1)
data['ar']['reserve_diff'] = avg_reserv['reserve_diff']                            
    
data['hol']['visit_date'] = pd.to_datetime(data['hol']['visit_date'])
data['hol']['visit_date'] = data['hol']['visit_date'].dt.date

prep_df = pd.merge(data['tra'], data['ar'],  how='inner', on=['air_store_id', 'visit_date'])
prep_df = pd.merge(prep_df,     data['as'],  how='inner', on='air_store_id')
prep_df = pd.merge(prep_df,     data['hol'], how='left',  on='visit_date')

In [48]:
prep_df

,air_store_id,visit_date,visitors,visit_datetime,reserve_visitors,reserve_diff,air_genre_name,air_area_name,latitude,longitude,day_of_week,holiday_flg
0,air_35512c42db0868da,2016-02-27,6,2016-02-27,12,0.000000,Dining bar,Tōkyō-to Musashino-shi Midorichō,35.717784,139.566260,Saturday,0
1,air_ee3a01f0c71a769f,2016-01-04,61,2016-01-04,2,0.000000,Cafe/Sweets,Shizuoka-ken Hamamatsu-shi Motoshirochō,34.710895,137.725940,Monday,0
2,air_ee3a01f0c71a769f,2016-01-08,21,2016-01-08,3,2.000000,Cafe/Sweets,Shizuoka-ken Hamamatsu-shi Motoshirochō,34.710895,137.725940,Friday,0
3,air_ee3a01f0c71a769f,2016-01-09,57,2016-01-09,25,2.250000,Cafe/Sweets,Shizuoka-ken Hamamatsu-shi Motoshirochō,34.710895,137.725940,Saturday,0
4,air_ee3a01f0c71a769f,2016-01-10,32,2016-01-10,5,2.000000,Cafe/Sweets,Shizuoka-ken Hamamatsu-shi Motoshirochō,34.710895,137.725940,Sunday,0
5,air_ee3a01f0c71a769f,2016-01-11,35,2016-01-11,6,2.666667,Cafe/Sweets,Shizuoka-ken Hamamatsu-shi Motoshirochō,34.710895,137.725940,Monday,1
6,air_ee3a01f0c71a769f,2016-01-16,34,2016-01-16,6,1.000000,Cafe/Sweets,Shizuoka-ken Hamamatsu-shi Motoshirochō,34.710895,137.725940,Saturday,0
7,air_ee3a01f0c71a769f,2016-01-17,58,2016-01-17,12,3.000000,Cafe/Sweets,Shizuoka-ken Hamamatsu-shi Motoshirochō,34.710895,137.725940,Sunday,0
8,air_ee3a01f0c71a769f,2016-01-18,14,2016-01-18,2,7.000000,Cafe/Sweets,Shizuoka-ken Hamamatsu-shi Motoshirochō,34.710895,137.725940,Monday,0
9,air_ee3a01f0c71a769f,2016-01-22,6,2016-01-22,1,9.000000,Cafe/Sweets,Shizuoka-ken Hamamatsu-shi Motoshirochō,34.710895,137.725940,Friday,0


In [49]:
# Encode fields:
prep_df['year']  = prep_df['visit_datetime'].dt.year
prep_df['month'] = prep_df['visit_datetime'].dt.month
prep_df['day']   = prep_df['visit_datetime'].dt.day
prep_df = prep_df.drop('visit_datetime', axis=1)   

# Encode labels of categorical columns:
cat_features = [col for col in ['air_genre_name', 'air_area_name', 'day_of_week', 'year']]
for column in cat_features:
    temp = pd.get_dummies(pd.Series(prep_df[column]))
    prep_df = pd.concat([prep_df,temp],axis=1)
    prep_df = prep_df.drop([column],axis=1)
prep_df['visitors'] = np.log1p(prep_df['visitors'])

In [50]:
air_ids = [air for air in prep_df['air_store_id'].unique()]
mult_series = dict()
scaler = MinMaxScaler(feature_range=(0, 1))

store_key = prep_df[['air_store_id', 'visit_date']]
prep_df = prep_df.drop(['air_store_id', 'visit_date'], axis=1)
cols = prep_df.columns
scaled_prep_df = pd.DataFrame(scaler.fit_transform(prep_df), columns=cols)
scaled_prep_df['air_store_id'] = store_key['air_store_id']
scaled_prep_df['visit_date'] = store_key['visit_date']

for air_id in air_ids:
    tmp = pd.DataFrame(scaled_prep_df[scaled_prep_df['air_store_id'] == air_id])
    tmp = tmp.drop('air_store_id', axis=1)
    tmp = tmp.set_index('visit_date')
    mult_series[str(air_id)] = tmp.astype('float32')

In [58]:
# From https://machinelearningmastery.com/convert-time-series-supervised-learning-problem-python/
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a list or NumPy array.
        n_in: Number of lag observations as input (X).
        n_out: Number of observations as output (y).
        dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # Input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # Forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # Put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # Drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

# Submissions are evaluated using RMSLE:
def RMSLE(y, pred):
    return mean_squared_error(y, pred)**0.5

In [59]:
mult_series['air_ee3a01f0c71a769f'].head()

,visitors,reserve_visitors,reserve_diff,latitude,longitude,holiday_flg,month,day,Asian,Bar/Cocktail,...,Ōsaka-fu Ōsaka-shi Ōhiraki,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,2016,2017
visit_date,,,,,,,,,,,,,,,,,,,,,
2016-01-04,0.714207,0.000613,0.000000,0.138678,0.529804,0.0,0.0,0.100000,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2016-01-08,0.498719,0.001225,0.005702,0.138678,0.529804,0.0,0.0,0.233333,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2016-01-09,0.700337,0.014706,0.006414,0.138678,0.529804,0.0,0.0,0.266667,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2016-01-10,0.583048,0.002451,0.005702,0.138678,0.529804,0.0,0.0,0.300000,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2016-01-11,0.601145,0.003064,0.007602,0.138678,0.529804,1.0,0.0,0.333333,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [67]:
# Convert data series for supervised learning:
tmp = pd.DataFrame(series_to_supervised(mult_series['air_ee3a01f0c71a769f'], 1, 1))
tmp.drop(tmp.columns[[i for i in range(104,206)]], axis=1, inplace=True)
super_data = tmp
for air_id in air_ids:
    tmp = series_to_supervised(mult_series[str(air_id)], 1, 1)
    # Drop columns that should not be predicted (column #103 is number of visitors:
    tmp.drop(tmp.columns[[i for i in range(104,206)]], axis=1, inplace=True)
    super_data = super_data.append(tmp)

In [72]:
super_data.groupby('visit_date').sum()

,var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t-1),var7(t-1),var8(t-1),var9(t-1),var10(t-1),...,var95(t-1),var96(t-1),var97(t-1),var98(t-1),var99(t-1),var100(t-1),var101(t-1),var102(t-1),var103(t-1),var1(t)
visit_date,,,,,,,,,,,,,,,,,,,,,
2016-01-02,0.456983,0.006127,0.000000,1.203180,1.538212,2.0,0.000000,0.000000,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.192263
2016-01-03,2.346712,0.073529,0.001629,1.125478,1.404228,3.0,0.000000,0.100000,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,2.093154
2016-01-04,4.451568,0.025123,0.002851,1.464769,2.932097,7.0,0.000000,0.400000,0.0,0.0,...,0.0,0.0,2.0,5.0,0.0,0.0,0.0,7.0,0.0,3.803718
2016-01-05,4.902782,0.031250,0.005702,2.163320,3.655486,2.0,0.000000,0.900000,0.0,0.0,...,0.0,8.0,1.0,1.0,0.0,0.0,0.0,10.0,0.0,4.436258
2016-01-06,7.492122,0.046569,0.017818,3.480856,6.763287,3.0,0.000000,1.633333,0.0,0.0,...,1.0,3.0,0.0,2.0,0.0,9.0,0.0,15.0,0.0,6.321769
2016-01-07,9.695975,0.103554,0.025216,4.638905,9.517995,0.0,0.000000,2.900000,0.0,0.0,...,0.0,7.0,0.0,0.0,0.0,4.0,10.0,21.0,0.0,8.397659
2016-01-08,14.481079,0.112132,0.044948,7.250254,14.391657,1.0,0.000000,5.733334,0.0,0.0,...,0.0,4.0,1.0,0.0,17.0,3.0,9.0,34.0,0.0,16.898600
2016-01-09,19.916286,0.219363,0.167052,10.653220,17.760580,1.0,0.000000,8.733334,0.0,1.0,...,29.0,0.0,0.0,1.0,5.0,3.0,3.0,41.0,0.0,22.087862
2016-01-10,18.242617,0.264706,0.130665,7.666118,13.743133,0.0,0.000000,8.566667,0.0,1.0,...,5.0,0.0,27.0,0.0,1.0,0.0,0.0,33.0,0.0,18.384926


In [74]:
# Prepare LSTM training, split up records into training and test data:
train_size = int(len(super_data) * 0.7)
test_size = len(super_data) - train_size

train = super_data[:train_size].values
test = super_data[train_size:].values

# Split into input and outputs
train_X, train_y = train[:,:-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]

# LSTM requires 3D data sets: [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(24850, 1, 103) (24850,) (10650, 1, 103) (10650,)


In [112]:
# Train model:
multi_model = Sequential()
batch_size = 32

multi_model.add(LSTM(4, batch_input_shape=(
    batch_size, train_X.shape[1], train_X.shape[2]), stateful=False))
# multi_model.add(Dense(1))
multi_model.add(Dense(32, activation='relu'))
multi_model.add(Dropout(0.5))
multi_model.add(Dense(1, activation='sigmoid'))

multi_model.compile(loss='mse', optimizer='adam')
for i in range(int(train_X.shape[0] / batch_size)):
    this_X = train_X[i*batch_size:(i+1)*batch_size][:][:]
    this_y = train_y[i*batch_size:(i+1)*batch_size]
    multi_history = multi_model.fit(this_X, this_y, epochs=20,
            batch_size =batch_size, verbose=2, shuffle=False)
    multi_model.reset_states()

Epoch 1/20
 - 1s - loss: 0.0270
Epoch 2/20
 - 0s - loss: 0.0269
Epoch 3/20
 - 0s - loss: 0.0259
Epoch 4/20
 - 0s - loss: 0.0259
Epoch 5/20
 - 0s - loss: 0.0259
Epoch 6/20
 - 0s - loss: 0.0255
Epoch 7/20
 - 0s - loss: 0.0242
Epoch 8/20
 - 0s - loss: 0.0243
Epoch 9/20
 - 0s - loss: 0.0253
Epoch 10/20
 - 0s - loss: 0.0248
Epoch 11/20
 - 0s - loss: 0.0236
Epoch 12/20
 - 0s - loss: 0.0240
Epoch 13/20
 - 0s - loss: 0.0237
Epoch 14/20
 - 0s - loss: 0.0237
Epoch 15/20
 - 0s - loss: 0.0225
Epoch 16/20
 - 0s - loss: 0.0221
Epoch 17/20
 - 0s - loss: 0.0226
Epoch 18/20
 - 0s - loss: 0.0219
Epoch 19/20
 - 0s - loss: 0.0217
Epoch 20/20
 - 0s - loss: 0.0206
Epoch 1/20
 - 0s - loss: 0.0184
Epoch 2/20
 - 0s - loss: 0.0176
Epoch 3/20
 - 0s - loss: 0.0186
Epoch 4/20
 - 0s - loss: 0.0176
Epoch 5/20
 - 0s - loss: 0.0168
Epoch 6/20
 - 0s - loss: 0.0168
Epoch 7/20
 - 0s - loss: 0.0175
Epoch 8/20
 - 0s - loss: 0.0177
Epoch 9/20
 - 0s - loss: 0.0166
Epoch 10/20
 - 0s - loss: 0.0159
Epoch 11/20
 - 0s - loss: 0.

Epoch 13/20
 - 0s - loss: 0.0193
Epoch 14/20
 - 0s - loss: 0.0191
Epoch 15/20
 - 0s - loss: 0.0194
Epoch 16/20
 - 0s - loss: 0.0207
Epoch 17/20
 - 0s - loss: 0.0200
Epoch 18/20
 - 0s - loss: 0.0203
Epoch 19/20
 - 0s - loss: 0.0202
Epoch 20/20
 - 0s - loss: 0.0204
Epoch 1/20
 - 0s - loss: 0.0126
Epoch 2/20
 - 0s - loss: 0.0122
Epoch 3/20
 - 0s - loss: 0.0123
Epoch 4/20
 - 0s - loss: 0.0120
Epoch 5/20
 - 0s - loss: 0.0112
Epoch 6/20
 - 0s - loss: 0.0112
Epoch 7/20
 - 0s - loss: 0.0108
Epoch 8/20
 - 0s - loss: 0.0104
Epoch 9/20
 - 0s - loss: 0.0104
Epoch 10/20
 - 0s - loss: 0.0104
Epoch 11/20
 - 0s - loss: 0.0104
Epoch 12/20
 - 0s - loss: 0.0107
Epoch 13/20
 - 0s - loss: 0.0106
Epoch 14/20
 - 0s - loss: 0.0096
Epoch 15/20
 - 0s - loss: 0.0100
Epoch 16/20
 - 0s - loss: 0.0092
Epoch 17/20
 - 0s - loss: 0.0087
Epoch 18/20
 - 0s - loss: 0.0093
Epoch 19/20
 - 0s - loss: 0.0092
Epoch 20/20
 - 0s - loss: 0.0087
Epoch 1/20
 - 0s - loss: 0.0107
Epoch 2/20
 - 0s - loss: 0.0105
Epoch 3/20
 - 0s - lo

Epoch 5/20
 - 0s - loss: 0.0681
Epoch 6/20
 - 0s - loss: 0.0648
Epoch 7/20
 - 0s - loss: 0.0612
Epoch 8/20
 - 0s - loss: 0.0615
Epoch 9/20
 - 0s - loss: 0.0579
Epoch 10/20
 - 0s - loss: 0.0515
Epoch 11/20
 - 0s - loss: 0.0510
Epoch 12/20
 - 0s - loss: 0.0512
Epoch 13/20
 - 0s - loss: 0.0491
Epoch 14/20
 - 0s - loss: 0.0444
Epoch 15/20
 - 0s - loss: 0.0392
Epoch 16/20
 - 0s - loss: 0.0377
Epoch 17/20
 - 0s - loss: 0.0360
Epoch 18/20
 - 0s - loss: 0.0413
Epoch 19/20
 - 0s - loss: 0.0364
Epoch 20/20
 - 0s - loss: 0.0344
Epoch 1/20
 - 0s - loss: 0.0606
Epoch 2/20
 - 0s - loss: 0.0547
Epoch 3/20
 - 0s - loss: 0.0618
Epoch 4/20
 - 0s - loss: 0.0667
Epoch 5/20
 - 0s - loss: 0.0633
Epoch 6/20
 - 0s - loss: 0.0593
Epoch 7/20
 - 0s - loss: 0.0630
Epoch 8/20
 - 0s - loss: 0.0685
Epoch 9/20
 - 0s - loss: 0.0631
Epoch 10/20
 - 0s - loss: 0.0607
Epoch 11/20
 - 0s - loss: 0.0562
Epoch 12/20
 - 0s - loss: 0.0627
Epoch 13/20
 - 0s - loss: 0.0560
Epoch 14/20
 - 0s - loss: 0.0576
Epoch 15/20
 - 0s - loss

Epoch 17/20
 - 0s - loss: 0.0107
Epoch 18/20
 - 0s - loss: 0.0082
Epoch 19/20
 - 0s - loss: 0.0099
Epoch 20/20
 - 0s - loss: 0.0090
Epoch 1/20
 - 0s - loss: 0.0214
Epoch 2/20
 - 0s - loss: 0.0204
Epoch 3/20
 - 0s - loss: 0.0213
Epoch 4/20
 - 0s - loss: 0.0209
Epoch 5/20
 - 0s - loss: 0.0236
Epoch 6/20
 - 0s - loss: 0.0195
Epoch 7/20
 - 0s - loss: 0.0199
Epoch 8/20
 - 0s - loss: 0.0226
Epoch 9/20
 - 0s - loss: 0.0206
Epoch 10/20
 - 0s - loss: 0.0195
Epoch 11/20
 - 0s - loss: 0.0199
Epoch 12/20
 - 0s - loss: 0.0204
Epoch 13/20
 - 0s - loss: 0.0204
Epoch 14/20
 - 0s - loss: 0.0209
Epoch 15/20
 - 0s - loss: 0.0190
Epoch 16/20
 - 0s - loss: 0.0184
Epoch 17/20
 - 0s - loss: 0.0184
Epoch 18/20
 - 0s - loss: 0.0174
Epoch 19/20
 - 0s - loss: 0.0197
Epoch 20/20
 - 0s - loss: 0.0191
Epoch 1/20
 - 0s - loss: 0.0141
Epoch 2/20
 - 0s - loss: 0.0117
Epoch 3/20
 - 0s - loss: 0.0130
Epoch 4/20
 - 0s - loss: 0.0124
Epoch 5/20
 - 0s - loss: 0.0130
Epoch 6/20
 - 0s - loss: 0.0108
Epoch 7/20
 - 0s - loss: 

Epoch 9/20
 - 0s - loss: 0.0280
Epoch 10/20
 - 0s - loss: 0.0297
Epoch 11/20
 - 0s - loss: 0.0283
Epoch 12/20
 - 0s - loss: 0.0283
Epoch 13/20
 - 0s - loss: 0.0269
Epoch 14/20
 - 0s - loss: 0.0295
Epoch 15/20
 - 0s - loss: 0.0266
Epoch 16/20
 - 0s - loss: 0.0278
Epoch 17/20
 - 0s - loss: 0.0298
Epoch 18/20
 - 0s - loss: 0.0288
Epoch 19/20
 - 0s - loss: 0.0288
Epoch 20/20
 - 0s - loss: 0.0269
Epoch 1/20
 - 0s - loss: 0.0233
Epoch 2/20
 - 0s - loss: 0.0220
Epoch 3/20
 - 0s - loss: 0.0219
Epoch 4/20
 - 0s - loss: 0.0200
Epoch 5/20
 - 0s - loss: 0.0218
Epoch 6/20
 - 0s - loss: 0.0216
Epoch 7/20
 - 0s - loss: 0.0215
Epoch 8/20
 - 0s - loss: 0.0234
Epoch 9/20
 - 0s - loss: 0.0207
Epoch 10/20
 - 0s - loss: 0.0210
Epoch 11/20
 - 0s - loss: 0.0204
Epoch 12/20
 - 0s - loss: 0.0226
Epoch 13/20
 - 0s - loss: 0.0227
Epoch 14/20
 - 0s - loss: 0.0228
Epoch 15/20
 - 0s - loss: 0.0227
Epoch 16/20
 - 0s - loss: 0.0227
Epoch 17/20
 - 0s - loss: 0.0201
Epoch 18/20
 - 0s - loss: 0.0253
Epoch 19/20
 - 0s - 

...

Epoch 4/20
 - 0s - loss: 0.0040
Epoch 5/20
 - 0s - loss: 0.0043
Epoch 6/20
 - 0s - loss: 0.0042
Epoch 7/20
 - 0s - loss: 0.0041
Epoch 8/20
 - 0s - loss: 0.0046
Epoch 9/20
 - 0s - loss: 0.0031
Epoch 10/20
 - 0s - loss: 0.0039
Epoch 11/20
 - 0s - loss: 0.0039
Epoch 12/20
 - 0s - loss: 0.0036
Epoch 13/20
 - 0s - loss: 0.0036
Epoch 14/20
 - 0s - loss: 0.0034
Epoch 15/20
 - 0s - loss: 0.0038
Epoch 16/20
 - 0s - loss: 0.0039
Epoch 17/20
 - 0s - loss: 0.0036
Epoch 18/20
 - 0s - loss: 0.0034
Epoch 19/20
 - 0s - loss: 0.0038
Epoch 20/20
 - 0s - loss: 0.0036
Epoch 1/20
 - 0s - loss: 0.0139
Epoch 2/20
 - 0s - loss: 0.0123
Epoch 3/20
 - 0s - loss: 0.0140
Epoch 4/20
 - 0s - loss: 0.0141
Epoch 5/20
 - 0s - loss: 0.0127
Epoch 6/20
 - 0s - loss: 0.0139
Epoch 7/20
 - 0s - loss: 0.0125
Epoch 8/20
 - 0s - loss: 0.0209
Epoch 3/20
 - 0s - loss: 0.0204
Epoch 4/20
 - 0s - loss: 0.0201
Epoch 5/20
 - 0s - loss: 0.0203
Epoch 6/20
 - 0s - loss: 0.0197
Epoch 7/20
 - 0s - loss: 0.0189
Epoch 8/20
 - 0s - loss: 0.01

Epoch 10/20
 - 0s - loss: 0.0170
Epoch 11/20
 - 0s - loss: 0.0155
Epoch 12/20
 - 0s - loss: 0.0154
Epoch 13/20
 - 0s - loss: 0.0135
Epoch 14/20
 - 0s - loss: 0.0121
Epoch 15/20
 - 0s - loss: 0.0139
Epoch 16/20
 - 0s - loss: 0.0130
Epoch 17/20
 - 0s - loss: 0.0100
Epoch 18/20
 - 0s - loss: 0.0100
Epoch 19/20
 - 0s - loss: 0.0138
Epoch 20/20
 - 0s - loss: 0.0130
Epoch 1/20
 - 0s - loss: 0.0144
Epoch 2/20
 - 0s - loss: 0.0168
Epoch 3/20
 - 0s - loss: 0.0121
Epoch 4/20
 - 0s - loss: 0.0156
Epoch 5/20
 - 0s - loss: 0.0163
Epoch 6/20
 - 0s - loss: 0.0127
Epoch 7/20
 - 0s - loss: 0.0165
Epoch 8/20
 - 0s - loss: 0.0168
Epoch 9/20
 - 0s - loss: 0.0150
Epoch 10/20
 - 0s - loss: 0.0147
Epoch 11/20
 - 0s - loss: 0.0183
Epoch 12/20
 - 0s - loss: 0.0133
Epoch 13/20
 - 0s - loss: 0.0134
Epoch 14/20
 - 0s - loss: 0.0151
Epoch 15/20
 - 0s - loss: 0.0158
Epoch 16/20
 - 0s - loss: 0.0156
Epoch 17/20
 - 0s - loss: 0.0131
Epoch 18/20
 - 0s - loss: 0.0159
Epoch 19/20
 - 0s - loss: 0.0154
Epoch 20/20
 - 0s -

Epoch 1/20
 - 0s - loss: 0.0478
Epoch 2/20
 - 0s - loss: 0.0472
Epoch 3/20
 - 0s - loss: 0.0468
Epoch 4/20
 - 0s - loss: 0.0451
Epoch 5/20
 - 0s - loss: 0.0448
Epoch 6/20
 - 0s - loss: 0.0442
Epoch 7/20
 - 0s - loss: 0.0416
Epoch 8/20
 - 0s - loss: 0.0393
Epoch 9/20
 - 0s - loss: 0.0389
Epoch 10/20
 - 0s - loss: 0.0380
Epoch 11/20
 - 0s - loss: 0.0342
Epoch 12/20
 - 0s - loss: 0.0336
Epoch 13/20
 - 0s - loss: 0.0297
Epoch 14/20
 - 0s - loss: 0.0297
Epoch 15/20
 - 0s - loss: 0.0290
Epoch 16/20
 - 0s - loss: 0.0249
Epoch 17/20
 - 0s - loss: 0.0263
Epoch 18/20
 - 0s - loss: 0.0227
Epoch 19/20
 - 0s - loss: 0.0200
Epoch 20/20
 - 0s - loss: 0.0197
Epoch 1/20
 - 0s - loss: 0.0122
Epoch 2/20
 - 0s - loss: 0.0117
Epoch 3/20
 - 0s - loss: 0.0092
Epoch 4/20
 - 0s - loss: 0.0119
Epoch 5/20
 - 0s - loss: 0.0118
Epoch 6/20
 - 0s - loss: 0.0085
Epoch 7/20
 - 0s - loss: 0.0083
Epoch 8/20
 - 0s - loss: 0.0079
Epoch 9/20
 - 0s - loss: 0.0070
Epoch 10/20
 - 0s - loss: 0.0073
Epoch 11/20
 - 0s - loss: 0.

Epoch 11/20
 - 0s - loss: 0.0305
Epoch 12/20
 - 0s - loss: 0.0287
Epoch 13/20
 - 0s - loss: 0.0270
Epoch 14/20
 - 0s - loss: 0.0265
Epoch 15/20
 - 0s - loss: 0.0264
Epoch 16/20
 - 0s - loss: 0.0247
Epoch 17/20
 - 0s - loss: 0.0246
Epoch 18/20
 - 0s - loss: 0.0241
Epoch 19/20
 - 0s - loss: 0.0233
Epoch 20/20
 - 0s - loss: 0.0227
Epoch 1/20
 - 0s - loss: 0.0171
Epoch 2/20
 - 0s - loss: 0.0170
Epoch 3/20
 - 0s - loss: 0.0164
Epoch 4/20
 - 0s - loss: 0.0164
Epoch 5/20
 - 0s - loss: 0.0158
Epoch 6/20
 - 0s - loss: 0.0159
Epoch 7/20
 - 0s - loss: 0.0157
Epoch 8/20
 - 0s - loss: 0.0155
Epoch 9/20
 - 0s - loss: 0.0152
Epoch 10/20
 - 0s - loss: 0.0154
Epoch 11/20
 - 0s - loss: 0.0151
Epoch 12/20
 - 0s - loss: 0.0149
Epoch 13/20
 - 0s - loss: 0.0148
Epoch 14/20
 - 0s - loss: 0.0147
Epoch 15/20
 - 0s - loss: 0.0146
Epoch 16/20
 - 0s - loss: 0.0145
Epoch 17/20
 - 0s - loss: 0.0144
Epoch 18/20
 - 0s - loss: 0.0143
Epoch 19/20
 - 0s - loss: 0.0143
Epoch 20/20
 - 0s - loss: 0.0141
Epoch 1/20
 - 0s - 

 - 0s - loss: 0.0057
Epoch 12/20
 - 0s - loss: 0.0065
Epoch 13/20
 - 0s - loss: 0.0098
Epoch 14/20
 - 0s - loss: 0.0062
Epoch 15/20
 - 0s - loss: 0.0071
Epoch 16/20
 - 0s - loss: 0.0087
Epoch 17/20
 - 0s - loss: 0.0079
Epoch 18/20
 - 0s - loss: 0.0074
Epoch 19/20
 - 0s - loss: 0.0071
Epoch 20/20
 - 0s - loss: 0.0073
Epoch 1/20
 - 0s - loss: 0.0068
Epoch 2/20
 - 0s - loss: 0.0066
Epoch 3/20
 - 0s - loss: 0.0042
Epoch 4/20
 - 0s - loss: 0.0055
Epoch 5/20
 - 0s - loss: 0.0070
Epoch 6/20
 - 0s - loss: 0.0069
Epoch 7/20
 - 0s - loss: 0.0059
Epoch 8/20
 - 0s - loss: 0.0052
Epoch 9/20
 - 0s - loss: 0.0075
Epoch 10/20
 - 0s - loss: 0.0034
Epoch 11/20
 - 0s - loss: 0.0072
Epoch 12/20
 - 0s - loss: 0.0073
Epoch 13/20
 - 0s - loss: 0.0047
Epoch 14/20
 - 0s - loss: 0.0083
Epoch 15/20
 - 0s - loss: 0.0069
Epoch 16/20
 - 0s - loss: 0.0030
Epoch 17/20
 - 0s - loss: 0.0055
Epoch 18/20
 - 0s - loss: 0.0060
Epoch 19/20
 - 0s - loss: 0.0054
Epoch 20/20
 - 0s - loss: 0.0031
Epoch 1/20
 - 0s - loss: 0.0065

In [114]:
# Make predictions:
y_pred = np.zeros((test_X.shape[0], 1))
for i in range(int(test_X.shape[0] / batch_size)):
    this_X = test_X[i*batch_size:(i + 1) * batch_size][:][:]
    this_pred = multi_model.predict(this_X, batch_size=batch_size)    
    y_pred[i*batch_size:(i + 1)*batch_size] = this_pred

test_X_nn = test_X.reshape((test_X.shape[0], test_X.shape[2]))
# Invert scaling for forecast as we used MinMaxScaler before on
# prep_df (scaler.fit_transform(prep_df)) so dimensions must match
inv_y_pred = np.concatenate((y_pred, test_X_nn[:, 1:]), axis=1)
inv_y_pred = scaler.inverse_transform(inv_y_pred)
inv_y_pred = inv_y_pred[:,0]
# Invert scaling for actual
test_y_nn = test_y.reshape((len(test_y), 1))
inv_y = np.concatenate((test_y_nn, test_X_nn[:, 1:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]

print(inv_y_pred[:10])
print(inv_y[:10])
rmsle = RMSLE(inv_y, inv_y_pred)
print('Test RMSLE: %.3f' % rmsle)

[ 3.33473672  3.25714592  3.22094182  2.97251855  3.10938202  3.22567851
  3.19243285  3.23780883  3.31044284  3.10299115]
[ 3.71357203  3.98898411  2.48490644  2.99573207  3.93182564  3.49650741
  4.27666616  4.84418678  2.3978951   3.04452252]
Test RMSLE: 0.687


In [104]:
y_pred

array([[ 0.55731946],
       [ 0.51856625],
       [ 0.51329488],
       ..., 
       [ 0.        ],
       [ 0.        ],
       [ 0.        ]])